# Options IV Calculation Test

This notebook tests the options data fetching and implied volatility (IV) calculation pipeline.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from adapters.options_adapter import OptionsAdapter
from adapters.rates_adapter import RatesAdapter
from adapters.ticker_adapter import TickerAdapter
from models.options_data import OptionsRequest, OptionType
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from engines.IV_smile import IVEngine


In [3]:
from update_rates import updateRates
updateRates()

FRED_API_KEY present: yes
Starting updateRates...
Starting rate updates...
Updated discount factors: added 1 new records
Updated SOFR: added 1 new records
Rate updates completed.
updateRates completed.
Last discount_factors date: 2025-12-12
SOFR update was included.


## Setup

Initialize the adapter and create the options request.


In [117]:
adapter = OptionsAdapter()

# Simulate duration selection (e.g. '1y' from frontend)
today = date.today()
expiry_end = today + relativedelta(weeks=16)

req = OptionsRequest(
    ticker="MSFT",
    optionType=OptionType.CALL,
    expiryStart=today,
    expiryEnd=expiry_end,
    strikeMin=420,
    strikeMax=550,
)


## Fetch Options Data


In [118]:
print(f"Requesting options for {req.ticker} until {expiry_end}...")
df = adapter.fetch_option_chain(req)
print(f"Fetched {len(df)} contracts.")



Requesting options for MSFT until 2026-04-07...
Fetched 250 contracts.


## Display Options Data


In [119]:
df[["optionType", "strike", "timeToExpiry", "midPrice", "expiry"]]

,optionType,strike,timeToExpiry,midPrice,expiry
0,call,420.0,0.008254,55.800,2025-12-19
1,call,425.0,0.008254,52.015,2025-12-19
2,call,430.0,0.008254,46.705,2025-12-19
3,call,432.5,0.008254,43.680,2025-12-19
4,call,435.0,0.008254,42.535,2025-12-19
...,...,...,...,...,...
245,call,520.0,0.257455,9.920,2026-03-20
246,call,525.0,0.257455,8.665,2026-03-20
247,call,530.0,0.257455,7.655,2026-03-20
248,call,540.0,0.257455,5.865,2026-03-20


## Calculate Implied Volatility


In [120]:
from engines.zero_rates import ZeroRatesEngine

df["rate"] = ZeroRatesEngine.interpolate_zero_rate(df, tte_col="timeToExpiry")

base_info = TickerAdapter.fetchBasic(req.ticker)
div = base_info.dividendYield
spot = base_info.spot
#
#print(f"Spot price: ${spot:.2f}")
#print(f"Risk-free rate: {rate:.4f}")
#print(f"Dividend yield: {div:.4f}%")
#print("\nCalculating IVs...")
#
surface_data = IVEngine.generateIVSmile(df, df["rate"], div, spot, OptionType.CALL) #type: ignore




In [121]:
print("\nIV Calculation Results:")
surface_data


IV Calculation Results:


,type,K,T,Price,rate,expiry,iv,S
0,call,420.0,0.008254,55.800,0.037541,2025-12-19,1.081857,476.325
1,call,425.0,0.008254,52.015,0.037541,2025-12-19,1.141309,476.325
2,call,430.0,0.008254,46.705,0.037541,2025-12-19,1.023608,476.325
3,call,432.5,0.008254,43.680,0.037541,2025-12-19,0.925930,476.325
4,call,435.0,0.008254,42.535,0.037541,2025-12-19,1.018287,476.325
...,...,...,...,...,...,...,...,...
245,call,520.0,0.257455,9.920,0.036098,2026-03-20,0.538661,476.325
246,call,525.0,0.257455,8.665,0.036098,2026-03-20,0.526017,476.325
247,call,530.0,0.257455,7.655,0.036098,2026-03-20,0.516511,476.325
248,call,540.0,0.257455,5.865,0.036098,2026-03-20,0.497604,476.325


In [122]:
import numpy as np
surface_data["F"] = spot * np.exp((surface_data["rate"] - div) * surface_data["T"])
surface_data["k"] = np.log(surface_data["K"] / surface_data["F"])
surface_data["w"] = surface_data["iv"] ** 2 * surface_data["T"]
surface_data[["w", "k", "expiry" ,"F"]]

,w,k,expiry,F
0,0.009660,-0.119800,2025-12-19,473.454046
1,0.010751,-0.107966,2025-12-19,473.454046
2,0.008648,-0.096270,2025-12-19,473.454046
3,0.007076,-0.090473,2025-12-19,473.454046
4,0.008558,-0.084709,2025-12-19,473.454046
...,...,...,...,...
245,0.074702,0.276675,2026-03-20,394.316551
246,0.071236,0.286244,2026-03-20,394.316551
247,0.068685,0.295723,2026-03-20,394.316551
248,0.063748,0.314415,2026-03-20,394.316551


In [123]:
theta = (surface_data.loc[surface_data["k"].abs().groupby(surface_data["T"]).idxmin()].set_index("T")["w"])
surface_data["theta"] = surface_data["T"].map(theta)
surface_data[["expiry", "T", "K", "w", "theta"]]


,expiry,T,K,w,theta
0,2025-12-19,0.008254,420.0,0.009660,0.001021
1,2025-12-19,0.008254,425.0,0.010751,0.001021
2,2025-12-19,0.008254,430.0,0.008648,0.001021
3,2025-12-19,0.008254,432.5,0.007076,0.001021
4,2025-12-19,0.008254,435.0,0.008558,0.001021
...,...,...,...,...,...
245,2026-03-20,0.257455,520.0,0.074702,0.248150
246,2026-03-20,0.257455,525.0,0.071236,0.248150
247,2026-03-20,0.257455,530.0,0.068685,0.248150
248,2026-03-20,0.257455,540.0,0.063748,0.248150


In [124]:
vega = IVEngine._vega(surface_data["iv"], surface_data["K"], surface_data["T"], surface_data["rate"], div, surface_data["F"])
surface_data["vega"] = np.clip(vega, 1e-6, None)
print(f"Spot price: ${spot:.2f}")
surface_data[["expiry", "K", "k", "vega"]]

Spot price: $476.32


,expiry,K,k,vega
0,2025-12-19,420.0,-0.119800,8.234866
1,2025-12-19,425.0,-0.107966,9.982360
2,2025-12-19,430.0,-0.096270,10.169333
3,2025-12-19,432.5,-0.090473,9.869749
4,2025-12-19,435.0,-0.084709,11.407885
...,...,...,...,...
245,2026-03-20,520.0,0.276675,19.180106
246,2026-03-20,525.0,0.286244,16.866012
247,2026-03-20,530.0,0.295723,14.958074
248,2026-03-20,540.0,0.314415,11.449441


In [125]:
def ssvi_w(k, theta, x):
    eta, rho = x
    phi = eta / np.sqrt(np.maximum(theta, 1e-12))
    w_ssvi = (
        1
        / 2
        * theta
        * (1 + rho * phi * k + np.sqrt((phi * k + rho) ** 2 + 1 - rho**2))
    )
    return w_ssvi


In [126]:
def objective(x, surface_data):
    eta, rho = x
    w_model = ssvi_w(surface_data["k"], surface_data["theta"], x)
    error = (w_model - surface_data["w"])
    loss = np.dot(error, error) 
    penalty = 1e3 * max(0.0, abs(rho) - 0.95)**2
    return loss + penalty

In [127]:
def butterfly_constraint(x):
    eta, rho = x
    return 2 - eta * (1 + abs(rho))


In [128]:
from scipy.optimize import minimize

eta = 0.5
rho = -0.3
x = np.array([eta, rho])
bounds = [(1e-5, None), (-0.999, 0.999)]

res = minimize(
    objective,
    x0=x,
    args=(surface_data,),
    method="SLSQP",
    bounds=bounds,
    constraints=[{
        'type': 'ineq',
        'fun': butterfly_constraint
    }]
)

eta_opt, rho_opt = res.x
print(f"Optimized eta: {eta_opt:.6f}, rho: {rho_opt:.6f}")

surface_data["w_ssvi"] = ssvi_w(surface_data["k"], surface_data["theta"], res.x)
error = objective(res.x, surface_data)
print(f"SSVI Loss: {error:.6f}")
w = surface_data["w"].values

err = np.abs(surface_data["w_ssvi"].values - w) # type: ignore
relative = err / np.maximum(w, 1e-6) # type: ignore
print(f"Mean Relative Error: {np.mean(relative)*100:.4f}%")


surface_data[["expiry", "K", "T", "w", "w_ssvi", "theta"]]

Optimized eta: 1.025584, rho: -0.950105
SSVI Loss: 0.043907
Mean Relative Error: 36.7275%


,expiry,K,T,w,w_ssvi,theta
0,2025-12-19,420.0,0.008254,0.009660,0.004828,0.001021
1,2025-12-19,425.0,0.008254,0.010751,0.004450,0.001021
2,2025-12-19,430.0,0.008254,0.008648,0.004077,0.001021
3,2025-12-19,432.5,0.008254,0.007076,0.003892,0.001021
4,2025-12-19,435.0,0.008254,0.008558,0.003708,0.001021
...,...,...,...,...,...,...
245,2026-03-20,520.0,0.257455,0.074702,0.117972,0.248150
246,2026-03-20,525.0,0.257455,0.071236,0.113779,0.248150
247,2026-03-20,530.0,0.257455,0.068685,0.109669,0.248150
248,2026-03-20,540.0,0.257455,0.063748,0.101704,0.248150


In [129]:
err = surface_data["w_ssvi"].values - surface_data["w"].values
rmse  = np.sqrt(np.mean(err**2))
nrmse = rmse / np.mean(surface_data["w"].values)
print("NRMSE:", nrmse)


NRMSE: 0.5176265224999401
